In [1]:
import sys
sys.path.append('/host/d/Github/')
import nibabel as nb
import glob
import os
import glob
import lpips
import torch
import numpy as np
import pandas as pd
from skimage.metrics import structural_similarity

import Diffusion_denoising_thin_slice.functions_collection as ff
import Diffusion_denoising_thin_slice.Build_lists.Build_list as Build_list
import Diffusion_denoising_thin_slice.Data_processing as Data_processing

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
build_sheet =  Build_list.Build_EM(os.path.join('/host/d/Data/minnie_EM/Patient_lists/minnie_EM_split_gaussian_simulation_v1.xlsx'))
batch_list, patient_id_list, _,_,simulation_file_1_list, simulation_file_2_list, ground_truth_file_list, slice_num_list = build_sheet.__build__(batch_list = ['test'])
n = ff.get_X_numbers_in_interval(total_number = patient_id_list.shape[0],start_number = 0,end_number = 1, interval = 1)
print('total test cases:', patient_id_list.shape[0])

total test cases: 7


In [3]:
def calc_mae_rmse_with_ref_window(img, ref, vmin, vmax):
    maes = []
    rmses = []
    for slice_num in range(0, img.shape[-1]):
        slice_img = img[:,:,slice_num]
        slice_ref = ref[:,:,slice_num]
        mask = np.where((slice_ref >= vmin) & (slice_ref <= vmax), 1, 0)
        # MAE
        mae = np.sum(np.abs(slice_img - slice_ref) * mask) / np.sum(mask)
        maes.append(mae)
        # RMSE
        rmse_value = np.sqrt( np.sum( (slice_img - slice_ref)**2 * mask) / np.sum(mask) )
        rmses.append(rmse_value)


    return np.mean(maes), np.std(maes), np.mean(rmses), np.std(rmses)

In [4]:
def calc_ssim_with_ref_window(img, ref, vmin, vmax):

    ssims = []
    for slice_num in range(0, img.shape[-1]):
        slice_img = img[:,:,slice_num]
        slice_ref = ref[:,:,slice_num]
        mask = np.where((slice_ref >= vmin) & (slice_ref <= vmax), 1, 0)
        _, ssim_map = structural_similarity(slice_img, slice_ref, data_range=vmax - vmin, full=True)
        ssim = np.sum(ssim_map * mask) / np.sum(mask)
        ssims.append(ssim)

    return np.mean(ssims), np.std(ssims)

In [5]:
def calc_lpips(imgs1, imgs2, vmin, vmax):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    loss_fn = lpips.LPIPS().to(device)
    
    lpipss = []
    for slice_num in range(0, imgs1.shape[-1]):
        slice1 = imgs1[:,:,slice_num]
        slice2 = imgs2[:,:,slice_num]

        slice1 = np.clip(slice1, vmin, vmax).astype(np.float32)
        slice2 = np.clip(slice2, vmin, vmax).astype(np.float32)

        slice1 = (slice1 - vmin) / (vmax - vmin) * 2 - 1
        slice2 = (slice2 - vmin) / (vmax - vmin) * 2 - 1

        slice1 = np.stack([slice1, slice1, slice1], axis=-1)
        slice2 = np.stack([slice2, slice2, slice2], axis=-1)
        # print('after stack, slice1 shape:', slice1.shape, ' slice2 shape:', slice2.shape)

        slice1 = np.transpose(slice1, (2, 0, 1))[np.newaxis, ...]
        slice2 = np.transpose(slice2, (2, 0, 1))[np.newaxis, ...]
        # print('after transpose, slice1 shape:', slice1.shape, ' slice2 shape:', slice2.shape)

        slice1 = torch.from_numpy(slice1).to(device)
        slice2 = torch.from_numpy(slice2).to(device)

        lpips_val = loss_fn(slice1, slice2)
        lpipss.append(lpips_val.item())

      

    return np.mean(lpipss), np.std(lpipss)


In [16]:
from scipy.ndimage import gaussian_filter
def add_noise_EMslice(x, seed, sigma1_range = [0.45, 0.55], sigma2_range = [1.8, 2.2], sigma_target_range = [0.2,0.4]):
    
    rng = np.random.default_rng(seed)
    eps = rng.standard_normal(size = x.shape).astype(np.float32)

    seed_sigma = seed + 1
    np.random.seed(seed_sigma)
    sigma1 = np.random.uniform(sigma1_range[0], sigma1_range[1])
    sigma2 = np.random.uniform(sigma2_range[0], sigma2_range[1])
    low1 = gaussian_filter(eps, sigma=sigma1)   # H_sigma1(eps)
    low2 = gaussian_filter(eps, sigma=sigma2)   # H_sigma2(eps)
    z = low1 - low2                             # z = (H_sigma1 - H_sigma2)(eps)

    sigma_target = np.random.uniform(sigma_target_range[0], sigma_target_range[1])
    alpha = sigma_target / (z.std() + 1e-8)

    y = np.copy(x) + alpha * z
    y = np.clip(y, 0.0, 1.0)
    return y, [z, sigma1, sigma2, sigma_target, alpha]

In [23]:
## metric calculations
results = []
for i in range(0,2):#n.shape[0]):
    patient_id = patient_id_list[n[i]]
    print(patient_id)

    # reference image
    gt_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_EM/pred_images', patient_id, 'epoch85_1/gt_img.nii.gz')
    gt_img = nb.load(gt_file).get_fdata()

    # noisy image
    condition_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_EM/pred_images', patient_id, 'epoch85_1/condition_img.nii.gz')
    condition_img = nb.load(condition_file).get_fdata()

    # noise2noise
    # noise2noise_file = os.path.join('/host/d/projects/denoising/models/noise2noise_simple_MR/pred_images_input_both', patient_id,'random_'+str(random_n), 'epoch85/pred_img.nii.gz')
    # noise2noise_img = nb.load(noise2noise_file).get_fdata()
    # noise2noise_img = np.transpose(noise2noise_img, (1,2,0))

    # # supervised method
    # supervised_file = os.path.join('/host/d/projects/denoising/models/unsupervised_MR_2/pred_images_input_both',patient_id,  'random_'+str(random_n), 'epoch/pred_img.nii.gz')
    # supervised_img = nb.load(supervised_file).get_fdata()

    # our method (unsupervised), beta = 0, 1 inference
    unsupervised_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_EM/pred_images', patient_id, 'epoch85_1/pred_img.nii.gz')
    unsupervised_img = nb.load(unsupervised_file).get_fdata()

    # our method (unsupervised), beta = 0, 10 inference
    unsupervised_avg10_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_EM/pred_images', patient_id, 'epoch85avg/pred_img_scans10.nii.gz')
    unsupervised_avg10_img = nb.load(unsupervised_avg10_file).get_fdata()


    # # our method (unsupervised), beta = 0, 20 inference
    unsupervised_avg20_file = os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_EM/pred_images', patient_id, 'epoch85avg/pred_img_scans20.nii.gz')
    unsupervised_avg20_img = nb.load(unsupervised_avg20_file).get_fdata()

    # let's try to generate 10-20 simulations
    final_image = np.zeros([20, gt_img.shape[0], gt_img.shape[1], gt_img.shape[2]])
    for ii in range(0,20):
        for slice_num in range(0, gt_img.shape[2]):
            seed = np.random.randint(0, 100000)
            x = gt_img[:,:,slice_num]
            y, _ = add_noise_EMslice(x, seed, sigma1_range = [0.50, 0.65], sigma2_range = [2,3], sigma_target_range = [0.3,0.4])
            final_image[ii, :, :, slice_num] = y
    
    real_avg10 = np.mean(final_image[0:10, :, :, :], axis = 0)
    print('real avg10 shape:', real_avg10.shape)
    real_avg20 = np.mean(final_image, axis = 0)
    # save
    nb.save(nb.Nifti1Image(real_avg10, affine = np.eye(4)), os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_EM/pred_images', patient_id, 'real_avg10.nii.gz'))
    nb.save(nb.Nifti1Image(real_avg20, affine = np.eye(4)), os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_EM/pred_images', patient_id, 'real_avg20.nii.gz'))

    ## calculate metrics
    # MAE
    vmin = 0
    vmax = 1
    mae_condition, mae_condition_std, rmse_condition, rmse_condition_std = calc_mae_rmse_with_ref_window(condition_img, gt_img, vmin, vmax)
    # mae_noise2noise, mae_noise2noise_std , rmse_noise2noise, rmse_noise2noise_std= calc_mae_rmse_with_ref_window(noise2noise_img, gt_img, vmin, vmax)
    # mae_supervised, mae_supervised_std, rmse_supervised, rmse_supervised_std = calc_mae_rmse_with_ref_window(supervised_img, gt_img, vmin, vmax)
    mae_unsupervised, mae_unsupervised_std, rmse_unsupervised, rmse_unsupervised_std = calc_mae_rmse_with_ref_window(unsupervised_img, gt_img, vmin, vmax)
    mae_unsupervised_avg10, mae_unsupervised_avg10_std, rmse_unsupervised_avg10, rmse_unsupervised_avg10_std = calc_mae_rmse_with_ref_window(unsupervised_avg10_img, gt_img, vmin, vmax)
    mae_unsupervised_avg20, mae_unsupervised_avg20_std, rmse_unsupervised_avg20, rmse_unsupervised_avg20_std = calc_mae_rmse_with_ref_window(unsupervised_avg20_img, gt_img, vmin, vmax)
    mae_real_avg10, mae_real_avg10_std, rmse_real_avg10, rmse_real_avg10_std = calc_mae_rmse_with_ref_window(real_avg10, gt_img, vmin, vmax)
    mae_real_avg20, mae_real_avg20_std, rmse_real_avg20, rmse_real_avg20_std = calc_mae_rmse_with_ref_window(real_avg20, gt_img, vmin, vmax)
    
    
    # # SSIM
    ssim_condition, ssim_condition_std = calc_ssim_with_ref_window(condition_img, gt_img, vmin, vmax)
    # ssim_noise2noise, ssim_noise2noise_std = calc_ssim_with_ref_window(noise2noise_img, gt_img, vmin, vmax)
    # ssim_supervised, ssim_supervised_std = calc_ssim_with_ref_window(supervised_img, gt_img, vmin, vmax)
    ssim_unsupervised, ssim_unsupervised_std = calc_ssim_with_ref_window(unsupervised_img, gt_img, vmin, vmax)
    ssim_unsupervised_avg10, ssim_unsupervised_avg10_std = calc_ssim_with_ref_window(unsupervised_avg10_img, gt_img, vmin, vmax)
    ssim_unsupervised_avg20, ssim_unsupervised_avg20_std = calc_ssim_with_ref_window(unsupervised_avg20_img, gt_img, vmin, vmax)
    ssim_real_avg10, ssim_real_avg10_std = calc_ssim_with_ref_window(real_avg10, gt_img, vmin, vmax)
    ssim_real_avg20, ssim_real_avg20_std = calc_ssim_with_ref_window(real_avg20, gt_img, vmin, vmax)
    

    # # lpips
    lpips_condition, _ = calc_lpips(condition_img, gt_img, vmin, vmax)
    # lpips_noise2noise, _ = calc_lpips(noise2noise_img, gt_img, vmin, vmax)
    # lpips_supervised, _ = calc_lpips(supervised_img, gt_img, vmin, vmax)
    lpips_unsupervised, _ = calc_lpips(unsupervised_img, gt_img, vmin, vmax)
    lpips_unsupervised_avg10, _ = calc_lpips(unsupervised_avg10_img, gt_img, vmin, vmax)
    lpips_unsupervised_avg20, _ = calc_lpips(unsupervised_avg20_img, gt_img, vmin, vmax)
    lpips_real_avg10, _ = calc_lpips(real_avg10, gt_img, vmin, vmax)
    lpips_real_avg20, _ = calc_lpips(real_avg20, gt_img, vmin, vmax)

    print('condition: ', mae_condition, rmse_condition, ssim_condition, lpips_condition)
    # print('noise2noise: ', mae_noise2noise, rmse_noise2noise, ssim_noise2noise, lpips_noise2noise)
    print('unsupervised: ', mae_unsupervised, rmse_unsupervised, ssim_unsupervised, lpips_unsupervised)
    print('unsupervised avg10: ', mae_unsupervised_avg10, rmse_unsupervised_avg10, ssim_unsupervised_avg10, lpips_unsupervised_avg10)
    print('unsupervised avg20: ', mae_unsupervised_avg20, rmse_unsupervised_avg20, ssim_unsupervised_avg20, lpips_unsupervised_avg20)
    print('real avg10: ', mae_real_avg10, rmse_real_avg10, ssim_real_avg10, lpips_real_avg10)
    print('real avg20: ', mae_real_avg20, rmse_real_avg20, ssim_real_avg20, lpips_real_avg20)

    results.append([patient_id, 
                    mae_condition,mae_unsupervised, mae_unsupervised_avg10,mae_unsupervised_avg20, mae_real_avg10, mae_real_avg20,
                    rmse_condition,rmse_unsupervised, rmse_unsupervised_avg10, rmse_unsupervised_avg20, rmse_real_avg10, rmse_real_avg20,
                    ssim_condition,ssim_unsupervised, ssim_unsupervised_avg10, ssim_unsupervised_avg20, ssim_real_avg10, ssim_real_avg20,
                    lpips_condition,lpips_unsupervised, lpips_unsupervised_avg10, lpips_unsupervised_avg20, lpips_real_avg10, lpips_real_avg20
                    ])
    dd = pd.DataFrame(results, columns = ['patient_id',
                                        'mae_condition','mae_unsupervised','mae_unsupervised_avg10','mae_unsupervised_avg20', 'mae_real_avg10', 'mae_real_avg20',
                                        'rmse_condition','rmse_unsupervised','rmse_unsupervised_avg10','rmse_unsupervised_avg20', 'rmse_real_avg10', 'rmse_real_avg20',
                                        'ssim_condition','ssim_unsupervised','ssim_unsupervised_avg10','ssim_unsupervised_avg20', 'ssim_real_avg10', 'ssim_real_avg20',
                                        'lpips_condition','lpips_unsupervised','lpips_unsupervised_avg10','lpips_unsupervised_avg20', 'lpips_real_avg10', 'lpips_real_avg20'
                                          ])
                                     
    dd.to_excel('/host/d/projects/denoising/models/unsupervised_gaussian_EM/quantitative_results_85.xlsx', index = False)

patient_022
real avg10 shape: (769, 769, 5)
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: /usr/local/lib/python3.8/dist-packages/lpips/weights/v0.1/alex.pth
co

In [26]:
files = ff.find_all_target_files(['*/epoch115*'],os.path.join('/host/d/projects/denoising/models/unsupervised_gaussian_EM/pred_images'))
import shutil
for f in files:
    shutil.rmtree(f)